In [1]:
!pip install tensorflow pillow matplotlib scikit-learn


  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached pillow-12.1.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (52 kB)
  Using cached scikit_learn-1.8.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-2.4.2-cp312-cp312-manylinux_2_27_x86_64.manyl

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2


2026-02-18 15:34:03.157691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-18 15:34:03.199844: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-18 15:34:04.407394: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
DATASET_PATH = "dataset"
IMAGE_SIZE = (160,160)
BATCH_SIZE = 16


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)


Found 6400 files belonging to 4 classes.
Using 5120 files for training.
Found 6400 files belonging to 4 classes.
Using 1280 files for validation.


W0000 00:00:1771409047.881067   16060 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)


In [6]:
augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [7]:
base_model = MobileNetV2(
    input_shape=(160,160,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
inputs = base_model.input

x = augmentation(inputs)
x = base_model(x, training=False)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(4, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,948 (9.87 MB)

 Trainable params: 328,964 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 53s 156ms/step - accuracy: 0.5223 - loss: 1.0153 - val_accuracy: 0.5422 - val_loss: 0.9278
Epoch 2/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 49s 153ms/step - accuracy: 0.5551 - loss: 0.9227 - val_accuracy: 0.5352 - val_loss: 0.9342
Epoch 3/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 48s 150ms/step - accuracy: 0.5666 - loss: 0.9019 - val_accuracy: 0.5648 - val_loss: 0.8719
Epoch 4/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 49s 154ms/step - accuracy: 0.5736 - loss: 0.8843 - val_accuracy: 0.5727 - val_loss: 0.8619
Epoch 5/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 55s 172ms/step - accuracy: 0.5861 - loss: 0.8765 - val_accuracy: 0.5695 - val_loss: 0.8782


In [11]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 180s 508ms/step - accuracy: 0.5135 - loss: 1.1669 - val_accuracy: 0.5594 - val_loss: 0.8651
Epoch 2/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 172s 536ms/step - accuracy: 0.5510 - loss: 0.9625 - val_accuracy: 0.5695 - val_loss: 0.8840
Epoch 3/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 184s 574ms/step - accuracy: 0.5752 - loss: 0.8971 - val_accuracy: 0.5680 - val_loss: 0.8756
Epoch 4/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 188s 588ms/step - accuracy: 0.5920 - loss: 0.8660 - val_accuracy: 0.5703 - val_loss: 0.8795
Epoch 5/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 186s 582ms/step - accuracy: 0.5908 - loss: 0.8599 - val_accuracy: 0.5734 - val_loss: 0.8787


In [12]:
model.save("alzheimers_model")


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=alzheimers_model.

In [13]:
model.export("alzheimers_model")


INFO:tensorflow:Assets written to: alzheimers_model/assets


INFO:tensorflow:Assets written to: alzheimers_model/assets


Saved artifact at 'alzheimers_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  124320448021712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448022096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448022672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448021328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448024976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448025168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448023248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448024208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448025360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124320448026512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12432044802670

In [14]:
import os
print(os.listdir("alzheimers_model"))


['assets', '.ipynb_checkpoints', 'fingerprint.pb', 'saved_model.pb', 'variables']
